In [1]:
from shapely import geometry
import pandas as pd
import xarray as xr
from pathlib import Path

import sys
sys.path.append('./')
sys.path.append('../')
from phase_o_matic.download import download_era

In [4]:
params = [geometry.box(-116, 45, -115, 46), pd.to_datetime('2020-01-03T20:01:0000'), Path('/tmp')]
area, ts, out_dir = params
out_fp = download_era(date = ts, out_dir = out_dir, subset = area)

def test_download(self, params = params):
    """
    Test downloading era5 data
    """
    area, ts, out_dir = params

    out_fp = download_era(date = ts, out_dir = out_dir, subset = area)

    ds = xr.open_dataset(out_fp)

    self.assertEqual(ds.level.size, 37, "levels are not right size")
    self.assertEqual(ds.time[0], pd.to_datetime('2020-01-03T20:00:00.000000000'), "incorrect time data")
    self.assertEqual(ds['level'][10], 100, "spot checking pressure levels")
    self.assertEqual(ds.isel(level = 3)['q'].data.shape, (1, 5, 5), "checking data shape")
    self.assertTrue('z' in ds.data_vars, "geopotential heights present")
    self.assertTrue('t' in ds.data_vars, "Temperature present")
    self.assertTrue('q' in ds.data_vars, "Specific humidity present")

2023-05-10 14:10:25,060 INFO Welcome to the CDS
2023-05-10 14:10:25,061 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-05-10 14:10:25,525 INFO Request is completed
2023-05-10 14:10:25,526 INFO Downloading https://download-0002-clone.copernicus-climate.eu/cache-compute-0002/cache/data6/adaptor.mars.internal-1683668467.7999847-7393-19-715f3261-f895-48ba-adf4-1bb3996524e2.nc to /tmp/ERA5_2020-01-03T20:00_-116.00_45.00_-115.00_46.00.nc (7.4K)
2023-05-10 14:10:26,973 INFO Download rate 5.1K/s


In [3]:
from phase_o_matic.preprocess import check_start_end, interpolate_to_heights
import numpy as np
check_start_end(np.arange(0, 10), min_alt= 1, max_alt = 12)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 13]), False, True)

In [6]:
ds = xr.open_dataset(out_fp)

In [7]:
ds

<xarray.Dataset>
Dimensions:    (longitude: 5, latitude: 5, level: 37, time: 1)
Coordinates:
  * longitude  (longitude) float32 -116.0 -115.8 -115.5 -115.2 -115.0
  * latitude   (latitude) float32 46.0 45.75 45.5 45.25 45.0
  * level      (level) int32 1 2 3 5 7 10 20 30 ... 850 875 900 925 950 975 1000
  * time       (time) datetime64[ns] 2020-01-03T20:00:00
Data variables:
    z          (time, level, latitude, longitude) float32 ...
    t          (time, level, latitude, longitude) float32 ...
    r          (time, level, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-05-09 21:41:08 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...